### Extracting Sensor data

Remember to **ONLY** uncomment and run the next two cells if **USING** Google Colab

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [ ]:
# import sys
# sys.path.append('/content/gdrive/MyDrive/ACSE-9')

In [ ]:
from numpy import *
from math  import *
import sys, os
import numpy as np
!pip install vtk -q
import vtk
import vtktools
import matplotlib.pyplot as plt
import datetime, time
import pprint as pprint
import joblib

     |████████████████████████████████| 59.5 MB 42 kB/s 
     |████████████████████████████████| 495 kB 37.9 MB/s 
     |████████████████████████████████| 3.1 MB 40.7 MB/s 
     |████████████████████████████████| 74 kB 2.9 MB/s 
     |████████████████████████████████| 3.0 MB 56.2 MB/s 
     |████████████████████████████████| 251 kB 66.8 MB/s 
     |████████████████████████████████| 1.3 MB 66.4 MB/s 
     |████████████████████████████████| 142 kB 54.4 MB/s 
     |████████████████████████████████| 294 kB 46.6 MB/s 


Helper functions - supplied by Dr. Laetitia Mottet (supervisor)

In [ ]:
def ReadData(path,filename,extension):
    '''
      This functions sees the reading of the data within a text file
      Parameters
      ----------

      path : string
        The file path
      
      filename : string
        The filename of the text file

      extension : 
        File path extension

      Returns
      ---------

      output : list
        Data of the text file
    '''
    output = []

    sf = open(path+filename+extension, 'r')
    data = sf.readlines()

    for i in range(0,len(data)):
        x = str.split(data[i])
        y = [float(v) for v in x]
        output.append(y)
    return output

#-------------------------------------------------#
#-- Function to initialise vtk files             -#
#-------------------------------------------------#
def Initialisation(filename):
    '''
      This function initialises the vtk file
      Parameters
      ----------
    
      filename : string
        The filename of the vtk file

      Returns
      ---------

      ugrid 
      
    '''
    # Read file
    print ('     Read file')
    if filename[-4:] == ".vtu":
        gridreader=vtk.vtkXMLUnstructuredGridReader()
    elif filename[-5:] == ".pvtu":
        gridreader=vtk.vtkXMLPUnstructuredGridReader()
    gridreader.SetFileName(filename)
    gridreader.Update()
    ugrid=gridreader.GetOutput()

    return ugrid

#-------------------------------------------------#
#-- Function to initialise probe filter          -#
#-------------------------------------------------#
def InitialisePointData(ugrid, coordinates):

    # Initialise probe
    points = vtk.vtkPoints()
    points.SetDataTypeToDouble()

    # Create points to be extracted
    print ('     Gathering  coordinates')
    NrbPoints = 0
    for nodeID in range(len(coordinates)):
        NrbPoints += 1
        points.InsertNextPoint(coordinates[nodeID][0], coordinates[nodeID][1], coordinates[nodeID][2])

    print ('           Set points into data...')
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    probe = vtk.vtkProbeFilter()

    print ('           Map data into probe...', 'VTK version ::', vtk.vtkVersion.GetVTKMajorVersion(),'.', vtk.vtkVersion.GetVTKMinorVersion())
    if vtk.vtkVersion.GetVTKMajorVersion() <= 5:
        probe.SetInput(polydata)
        probe.SetSource(ugrid)
    else:
        probe.SetInputData(polydata)
        probe.SetSourceData(ugrid)

    probe.Update()
    
    return probe, points, NrbPoints

#-------------------------------------------------#
#-- Function to initialise cell filter           -#
#-------------------------------------------------#
def InitialiseCellLocator(ugrid):

    # Initialise locator
    print ('     Initialise cell Locator')
    CellLocator = vtk.vtkCellLocator()
    CellLocator.SetDataSet(ugrid)
    CellLocator.Update()
    
    return CellLocator

#-------------------------------------------------#
#-- Function to do the rotation                  -#
#-------------------------------------------------#
def Rotation(coord_old, theta, x_centre, y_centre):
    
    rotation = np.array([[np.cos(np.radians(theta)), -np.sin(np.radians(theta))],
                        [ np.sin(np.radians(theta)), np.cos(np.radians(theta))]])
    tmp = np.array(coord_old)[:,[0,1]] - [x_centre, y_centre]
    tmp = np.dot(tmp,rotation)
    tmp = tmp + [x_centre, y_centre]
    coord_new = np.append(tmp,np.array(coord_old)[:,[2]], axis=1)

    return coord_new

In [ ]:
#--------------------------------#
#-- Choose variables           --#
#--------------------------------#
# Vtu files
path      = 'ClarenceCentre/run_Clip_ToSend/'
# path      = '/content/gdrive/MyDrive/ACSE-9/ClarenceCentre/run_Clip_ToSend/'
extension = '.vtu'
name_simu = 'ClarenceCentre'
fieldname = 'CO2_ppm'
vtu_start = 0
vtu_end = 410
vtu_step  = 1


#--------------------------------#
#-- Geometry input variables   --#
#--------------------------------#

# Geometry variables
Pi = np.pi

theta = 110.0  # Angle of rotation

#-- Size of the building in which the room is
lz_box1 = 9.0  # Length in the z-direction
lx_box1 = 48.0 # Length in the x-direction
ly_box1 = 8.11 # Length in the y-direction

#-- Position of the building in which the room is
dx_box = 20.0 * lz_box1 # x Position
dy_box = 6.0  * lz_box1 # y Position
dz_box = 0.0            # z Position

e_room  = 0.5 # Thickness of the walls 50cm

#-- Centre of the building (needed for rotation)
x_centre = dx_box + lx_box1
y_centre = dy_box + ly_box1
print ('x_centre', x_centre, 'y_centre', y_centre)

#-- Parameters to define the room
x_room = 42.2    # x position of the left hand side corner
y_room = e_room  # y position of the left hand side corner
z_room = 6.08    # z position of the left hand side corner

#-- Dimensions of the room
lx_room = 5.3  # Length of the room
ly_room = 7.11 # Width of the room
lz_room = 2.42 # Height of the room

#-- Min and max coordiantes of the room - before rotation
xmin = dx_box+x_room
xmax = dx_box+x_room+lx_room
ymin = dy_box+y_room 
ymax = dy_box+y_room+ly_room 
zmin = dz_box+z_room
zmax = dz_box+z_room+lz_room

print ('Before Coordinate Rotation')
print ('xmin::', xmin, ' xmax::', xmax)
print ('ymin::', ymin, ' ymax::', ymax)
print ('zmin::', zmin, ' zmax::', zmax)

# --------------------------#
# -- Coordinates Fluidity --#
#---------------------------#
coordinates = []

tap1 = [xmax-1.0,  ymin+0.1,  zmin+0.9]
tap2 = [xmax-4.3,  ymin+0.1,  zmin+0.9]
tap3 = [xmax-1.7,  ymin+7.0,  zmin+0.9]
tap4 = [xmax-0.1 , ymin+4.14, zmin+1.2]
tap5 = [xmax-5.2 , ymin+3.14, zmin+1.22]
tap6 = [xmax-4.53, ymin+6.17, zmin+2.4]
tap7 = [xmax-2.65, ymin+4.14, zmin+0.79]

coord_init = np.array([tap1,  tap2,  tap3,  tap4,  tap5,  tap6,  tap7])

print ('\n Coordinates Concentration Init::')
print (coord_init)

coordinates = Rotation(coord_init, theta, x_centre, y_centre)

print ('\n Coordinates Concentration ::')
print (coordinates)

#-- Write Coordinates

# sys.exit('\n EXIT HERE')

#---------------------------------------------------------------------
# EXTRACT DATA
#---------------------------------------------------------------------
tic = time.time()

#-- CO2
CO2     = []

#-- Time
TimeVTU = []

r = 0

for vtuID in range(vtu_start,vtu_end,vtu_step):
    filename=path+name_simu+'_'+str(vtuID)+extension
    print ('\n\n  '+str(filename))

    CO2.append([])
    
    # Read file
    ugrid = Initialisation(filename)

    # Initialise probe
    probe, points, NrbPoints = InitialisePointData(ugrid, coordinates)

    # Initialise cell location
    CellLocator = InitialiseCellLocator(ugrid)

    #-- Check Validity of points
    print ('           Check point Validity')
    valid_ids = probe.GetOutput().GetPointData().GetArray('vtkValidPointMask')
    validPoints = vtktools.arr([valid_ids.GetTuple1(i) for i in range(NrbPoints)])
    print ('           ... ', len(validPoints)-np.sum(validPoints), 'invalid points')

    # Extract data
    print ('     Extract Data')
    for nodeID in range(len(coordinates)):
        # If valid point, extract using probe,
        # Otherwise extract the cell:
        #    If no cell associated - then it is really a non-valid point outside the domain
        #    Otherwise: do the average over the cell values - this provide the tracer value.
        # We need to do that as it is a well-known bug in vtk libraries - sometimes it returns an invalid node while it is not...
        if validPoints[nodeID] == 1:
            tmp = probe.GetOutput().GetPointData().GetArray(fieldname).GetValue(nodeID)
            CO2[r].append(tmp)
        else:
            coord_tmp = np.array(points.GetPoint(nodeID))
            cellID =  CellLocator.FindCell(coord_tmp) # cell ID which contains the node
            idlist=vtk.vtkIdList()
            ugrid.GetCellPoints(cellID, idlist)
            pointsID_to_cellID = np.array([idlist.GetId(k) for k in range(idlist.GetNumberOfIds())]) # give all the points asociated with this cell
            if len(pointsID_to_cellID) == 0: # Non-valid points - We assign negative value - like that we know we are outside the domain
                CO2[r].append(-1e20)
            else:
                tmp = 0
                for pointID in pointsID_to_cellID:
                    tmp += ugrid.GetPointData().GetArray(fieldname).GetTuple(pointID)[0]
                tmp = tmp/len(pointsID_to_cellID)
                CO2[r].append(tmp)
        # print("nodeID: ", valid_ids)

    # Time
    time_tmp = probe.GetOutput().GetPointData().GetArray('Time').GetValue(0)
    TimeVTU.append(time_tmp)
    print ('      Time ::', time_tmp)
    print ('      CO2  ::', CO2[r])

    r += 1

print ('\n Time in (s) ::')
print (TimeVTU)    
np.savetxt('TimeVTU.dat',TimeVTU)

print ('\n CO2 ::')
CO2 = np.array(CO2)
print (CO2)
CO2_all = [np.array(TimeVTU)]
CO2_all = np.transpose(np.append(CO2_all, np.transpose(CO2), axis = 0))

# joblib.dump(CO2, '/content/gdrive/MyDrive/ACSE-9/models/sensor_CO2_410.pkl') #CO2 data -> ( nSnapshot x 7(sensors) )
# joblib.dump(CO2_all, '/content/gdrive/MyDrive/ACSE-9/models/CO2_Fluidity_410.pkl') #with the time column
# joblib.dump(CO2, '/content/gdrive/MyDrive/ACSE-9/models/sensor_CO2_all.pkl') #CO2 data -> ALL time levels

toc = time.time()

print ('\n\nTime : ', toc - tic, 'sec')

x_centre 228.0 y_centre 62.11
Before Coordinate Rotation
xmin:: 222.2  xmax:: 227.5
ymin:: 54.5  ymax:: 61.61
zmin:: 6.08  zmax:: 8.5

 Coordinates Concentration Init::
[[226.5   54.6    6.98]
 [223.2   54.6    6.98]
 [225.8   61.5    6.98]
 [227.4   58.64   7.28]
 [222.3   57.64   7.3 ]
 [222.97  60.67   8.48]
 [224.85  58.64   6.87]]

 Coordinates Concentration ::
[[221.45593863  66.08811021   6.98      ]
 [222.58460511  69.18909586   6.98      ]
 [228.17923182  64.38595605   6.98      ]
 [224.94447869  63.86062547   7.28      ]
 [225.7490888   68.99507798   7.3       ]
 [228.36720395  67.32916289   8.48      ]
 [225.81663006  66.25684165   6.87      ]]


  /content/gdrive/MyDrive/ACSE-9/ClarenceCentre/run_Clip_ToSend/ClarenceCentre_0.vtu
     Read file
     Gathering  coordinates
           Set points into data...
           Map data into probe... VTK version :: 9 . 0
     Initialise cell Locator
           Check point Validity
           ...  0.0 invalid points
     Extract Data
no

KeyboardInterrupt: ignored

In [ ]:
print(type(CO2_all))
print(type(CO2))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
joblib.dump(CO2, 'output_files/sensor_CO2_410.pkl')

Import pod coeffs and do below process in concetenating pod_coeffs with CO2 as well as time_levels with pod_coeffs and CO2 

In [ ]:
pca_compress = joblib.load('output_files/pod_coefficients_410.pkl') 
CO2_sensor = joblib.load('output_files/sensor_CO2_410.pkl')

In [ ]:
print(pca_compress.shape)
print(CO2_sensor.shape)

(410, 43)
(410, 7)


Scaling of $CO_{2}$ between [-1, 1]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
'''
  Scaling sensor location CO2 values
'''

scaler_enhanced = MinMaxScaler((-1,1))
scaler_enhanced.fit(CO2_sensor)
norm_sensor_CO2 = scaler_enhanced.transform(CO2_sensor)
print("Standardised CO2 at sensor locations: ", norm_sensor_CO2)

Standardised CO2 at sensor locations:  [[ 1.          1.          1.         ...  1.          1.
   1.        ]
 [ 0.9737602   0.95008819  1.         ...  1.          1.
   1.        ]
 [-0.59716197 -0.82432065  1.         ...  1.          1.
   1.        ]
 ...
 [-0.99544474 -0.98879937 -0.99900326 ... -0.99823697 -0.96347561
  -0.9771921 ]
 [-0.99476439 -0.98835156 -0.9990045  ... -0.99919844 -0.98451205
  -0.97629523]
 [-0.9941548  -0.98801283 -0.99899582 ... -1.         -1.
  -0.97529972]]


In [ ]:
X_train_pca = pca_compress.copy()
print(X_train_pca.shape)
print(norm_sensor_CO2.shape, "\n")

(410, 43)
(410, 7) 



In [ ]:
'''
  Scaling compressed dataset (undergone pca)
'''
scaler_pca = MinMaxScaler((-1,1))
X_train_pca_scaled = scaler_pca.fit_transform(X_train_pca)
np.allclose(X_train_pca, scaler_pca.inverse_transform(X_train_pca_scaled))

-1.0


In [ ]:
'''
  Concatenating scaled 43 pod coefficients with the scaled sensor CO2 values
'''
enhanced_data_scaled = np.concatenate((X_train_pca_scaled, norm_sensor_CO2), axis=1)
print(enhanced_data_scaled.shape)

joblib.dump(enhanced_data_scaled, 'output_files/enhanced_data_scaled_410.pkl')
joblib.dump(scaler_pca, 'output_files/scaler_pca_410.pkl')
# joblib.dump(enhanced_data, '/content/gdrive/MyDrive/ACSE-9/models/enhanced_data_410.pkl')
joblib.dump(scaler_enhanced, '/content/gdrive/MyDrive/ACSE-9/models/CO2_scaler_minmax_410.pkl')

(410, 50)


['/content/gdrive/MyDrive/ACSE-9/models/scaler_pca_410.pkl']